In [15]:
import random
import itertools as it
#Initialization Table
table = [
    [1,2,3,4,3,2,1,5,3,1,2,3],
    [2,1,8,5,3,1,1,2,3,5,2,5],
    [5,4,2,1,3,3,4,1,1,8,2,8],
    ]

#transpose matrix and get Jobs ji [Ai,Li,Bi]
Jobs = list(zip(*table))

In [16]:
#check if the pair is nested and return P1
def check_nesting(j_1,j_2):
    if sum(j_2) <= j_1[1]: # L_j1 > a_j1 + L_j1 + b_j1
        return sum(j_1)
    return 0

#check if the pair is interleaved and return the sum a1 + P2
def check_interleaving(j_1,j_2):
    if j_2[0] <= j_1[1] and j_1[2] <= j_2[1]: # a_j2 <= L_j1 and b_j1 <= L_j2
        return j_1[0] + sum(j_2)
    return 0
         

In [17]:
#return the makespan of the sequence, check every pair if is nested or interleaved and get its sum
#if the pair is not nested or interleaved just return the sum of every job P1 and P2

def makespan(S):
    makespan = 0
    for i in range(int(len(S)/2)):
        if len(S)%2!=0 and i==int(len(S)/2): #if odd and the last job just return True
            makespan += sum(S[i])
        else:
            s1 = S[2*i]
            s2 = S[2*i+1]
            if check_nesting(s1,s2) != 0 or check_interleaving(s1,s2):
                makespan += check_nesting(s1,s2) + check_interleaving(s1,s2)
            else:
                makespan += sum(s1)+sum(s2)

    return makespan

In [18]:
#just a function to create a random sequence from a table
   
def generate_random_sequence(Jobs):
    job_rand = Jobs.copy()
    random.shuffle(job_rand)
    return job_rand

In [19]:

# Implementation of the paper 
# Li, H. and Zhao, H. (2007). “Scheduling Coupled-Tasks on a Single Machine”. IEEE Symposium on Computational
# Intelligence in Scheduling, 137-142.
# Author: Penailillo Nicolas

# CSFS(S). Let S be the sequence
# <(j1,j2, ..,jn)> . Start job j1 at time 0. For each index
# i= 2,3,...,n, scheldule job ji at the ealiest possible time in
# the partial schedule. Return the makespan of the constructed
# schedule 


def CSFS(S, return_S = False):
    partial_schedule = []
    partial_schedule.append(S[0])
    for i in range(1,len(S)):
        found_best_position = False
        for j in range(len(partial_schedule)):
            lowest = 100000
            if check_nesting(partial_schedule[j], S[i]) or check_interleaving(partial_schedule[j], S[i]):
                partial_schedule.insert(j+1, S[i])
                found_best_position = True
                break
        if not found_best_position:
            partial_schedule.append(S[i])
    
    
    if not return_S:
        return makespan(partial_schedule)
    else:
        return [S,makespan(partial_schedule)]

#This funtion returns M(S) =(j_1,j_[n/3],j_[2n/3],j_n,CSFS(S)).
def M(S,CSFS_value):
    return [S[0], S[int(len(S)/3)],S[int(2*len(S)/3)],S[len(S)-1],CSFS_value]

#This function return the neighborhood for the tabu search, see the paper details
def neighborhood(S,K):
    N=[]
    for i in range(len(S)):
        for k in range(K):
            s_copy = S.copy()
            subseq= S[i:i+k+1]

            s_copy[i:i+k+1]= []
            for m in range(len(s_copy)+1):
                s_copy_2 = s_copy.copy()
                s_copy_2[m:m]=subseq
                N.append(s_copy_2)

    return N

#this function iterate a list and search the minimal CSFS() of the element in the list
def best_neighbor_func(neighbors):
    best = []
    min = 100000000
    for neighbor in neighbors:
        if CSFS(neighbor) < min:
            best = neighbor
            min = CSFS(neighbor)
    return best

#tabu search, it iterates untill completes max_iterations without changes in S
def tabu(S,K,max_iterations):
    L=[]
    S_b = S
    C_b = CSFS(S)
    s__not_updated = 0
    iterations = 0
    while s__not_updated < max_iterations:
        iterations+=1
        neighbors = neighborhood(S,K)
        s__not_updated = 1000
        best_neighbor = best_neighbor_func(neighbors)
        new_csfs =CSFS(best_neighbor)
        Snew = []
        if M(best_neighbor,new_csfs) in L:
            Snew = generate_random_sequence(Jobs)
        else:
            Snew = best_neighbor
        
        if CSFS(Snew) < C_b:
            S_b = Snew
            C_b = CSFS(Snew)
            L.append(M(S,C_b))
            S=Snew
        else:
            s__not_updated+=1
    
    return(S_b)
        
    



In [64]:
S_optimal = tabu(generate_random_sequence(Jobs),3,1000)
print("Optimal makespan", CSFS(S_optimal))
print("Optimal sequence", CSFS(S_optimal,True)[0])

Optimal makespan 77
Optimal sequence [(1, 5, 8), (2, 2, 2), (3, 3, 3), (1, 2, 5), (3, 3, 1), (1, 1, 4), (2, 1, 4), (5, 2, 1), (3, 5, 8), (3, 8, 2), (2, 1, 3), (4, 5, 1)]
